# Using GLM though OpenAI SDK and Ecosystem

This notebook section will walk you through calling `GLM-4.5` models using the `OpenAI` SDK and supporting frameworks.
In this section you will learn how to call GLM-4.5 models using Langchain's `ChatOpenAI` class, and how to use the `OpenAI SDK` class to implement calling GLM-4 models.


## 1. Use OpenAI SDK

First, you need to import the `OpenAI` SDK and change the `OpenAI` `base_url` to `https://open.bigmodel.cn/api/paas/v4/` in order to call the `GLM-4` model. 
in order to call the `GLM-4` model. Once you have done this, you can use the `GLM-4` model by simply filling in the Key.

In [1]:
from openai import OpenAI

client = OpenAI(
    api_key="your api key of z.ai",
    base_url="https://open.bigmodel.cn/api/paas/v4/")

## 2. Simple use of OpenAI SDK to call GLM-4.5

Other examples (such as tool calls) can also be called in the same way as OpenAI.

In [2]:
response = client.chat.completions.create(
    model="glm-4.5",
    messages=[
        {
            "role": "user",
            "content": "tell me a joke"
        }
    ],
    top_p=0.7,
    temperature=0.9,
    stream=False,
    max_tokens=2000,
)
response.choices[0].message.content

"Sure! Here's a clean, punny one for you:\n\n**Why don't penguins fly?**  \n*Because they're not tall enough to be pilots!*  \n\n*(...ba-dum-tss!)* 😄\n\nWant another? I’ve got puns, dad jokes, or even a shaggy dog story if you’re brave!"

## 3. Using Langchain's ChatOpenAI class



Langchain's `ChatOpenAI` class is a wrapper around the `OpenAI` SDK, making it easier to call. Here we show how to use the `ChatOpenAI` class to call the `GLM-4.5` model.

In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4.5",
    openai_api_key="your api key of z.ai",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)
conversation.invoke({"question": "tell me a joke"})

/var/folders/cz/76xt997d6kl_rwv3_7mx_nm40000gn/T/ipykernel_59098/3774650906.py:28: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/var/folders/cz/76xt997d6kl_rwv3_7mx_nm40000gn/T/ipykernel_59098/3774650906.py:29: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation = LLMChain(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: tell me a joke

> Finished chain.


{'question': 'tell me a joke',
 'chat_history': [HumanMessage(content='tell me a joke', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", additional_kwargs={}, response_metadata={})],
 'text': "Why don't scientists trust atoms?\n\nBecause they make up everything!"}

## 4. Calling GLM-4.5 with LangChain AgentExecutor


GLM-4 also plugs nicely into LangChain's AgentExecutor, which shows how to call a `GLM-4.5` model using `AgentExecutor`.

In [5]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_tavily import TavilySearch

os.environ["TAVILY_API_KEY"] = "your api keys of tavily"
tools = [TavilySearch(max_results=2)]
prompt = hub.pull("hwchase17/react")
# Choose the LLM to use
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
Thought: I need to understand what LangChain is. Since this is a technical concept that might have recent developments, I should search for a comprehensive definition and explanation.

Action: tavily_search
Action Input: "what is LangChain"
Observation{'query': 'what is LangChain"\nObservation', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.reddit.com/r/LangChain/comments/18xjizi/difference_between_answer_and_observation/', 'title': 'Difference between Answer and Observation? - LangChain - Reddit', 'content': ": r/LangChain : r/LangChain Image 1: r/LangChain icon Go to LangChain r/LangChain Image 2: r/LangChain iconr/LangChain LangChain is an open-source framework and developer toolkit that helps developers get LLM applications from prototype to production. I am using a custom callback handler to do that, and I can stream the intermediate SQL query and everything, but not the 'Answer', or the 'Observa

{'input': 'what is LangChain?',
 'output': "LangChain is an open-source framework designed for developing applications powered by large language models (LLMs). It simplifies every stage of the LLM application lifecycle, from prototype to production. \n\nKey aspects of LangChain include:\n\n1. **Core Framework**: Provides building blocks, components, and third-party integrations for developing LLM applications\n2. **Cognitive Architecture**: Includes chains, agents, and retrieval strategies that form the application's reasoning capabilities\n3. **LangGraph**: A low-level orchestration framework for building stateful, multi-actor applications with LLMs by modeling steps as edges and nodes in a graph\n4. **Ecosystem Tools**:\n   - LangSmith: A unified observability and evaluation platform for debugging, testing, and monitoring AI applications\n   - LangGraph Platform: Deployment platform for scaling long-running, stateful workflows\n\nLangChain is trusted by major companies including Klar

## 5. Using OpenAI SDK to call CogView-4

We also support using the OpenAI interface to call the `CogView-4` model. Here we show how to use the `OpenAI SDK` class to call the `CogView-4` model. However, there are still many parameters that cannot be used.

In [6]:
response_cogview = client.images.generate(
    model="cogview-4",
    prompt="A scene of a city inside a crystal bottle"
)
response_cogview

ImagesResponse(created=1756970587, background=None, data=[Image(b64_json=None, revised_prompt=None, url='https://maas-watermark-prod.cn-wlcb.ufileos.com/1756970596375_watermark.png?UCloudPublicKey=TOKEN_75a9ae85-4f15-4045-940f-e94c0f82ae90&Signature=JzJpYkQXHzbcDnymM4p5780zgQI%3D&Expires=1757056996')], output_format=None, quality=None, size=None, usage=None)

In [8]:
from IPython.display import display, Image

display(Image(url=response_cogview.data[0].url))

## 6. UsingOpenAI SDK to call Embedding Model


We also support using the OpenAI interface to call the `Embedding` model. Here we show how to use the `OpenAI SDK` class to call the `Embedding` model.

Please note that the input of the `Embedding` model does not currently support lists and can only be used as strings. Here, I only show the first 10 values.

In [9]:
response_embedding = client.embeddings.create(
    model="embedding-3",
    input="hello",
)
response_embedding.data[0].embedding[:10]

[-0.0021513698,
 0.0056886184,
 -0.04387432,
 0.028451975,
 -0.025206413,
 0.019141711,
 0.0012881567,
 0.020764494,
 -0.0116200615,
 0.022091147]